In [1]:
import torch
from torch.utils.data import DataLoader

import numpy as np

from rpn.build_rpn import RPN_Model
from sam.build_sam import SAM_Model
from transformers import pipeline

from data_builder.build_dataset import PlanetscopeDataset

import os
import time
import json

from eval_utils import filter_boxes_predictions, filter_boxes, calculate_iou, calculate_precision_recall, filter_masks
from eval_utils import generate_random_bounding_boxes, generate_uniform_spaced_bounding_boxes

import pandas as pd
from rpn import _transforms as T
from torch.utils.data import ConcatDataset

/home/aghosh57/.conda/envs/dl4cv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_transform(image_enhancement="FALSE"):
    transforms = []
    # converts the image, a PIL image, into a PyTorch Tensor
    if image_enhancement == "TRUE":
        transforms.append(T.ContrastBasedAdaptiveGammaCorrection())
    transforms.append(T.ToTensor())
    return T.Compose(transforms)

#### Read and Print the evaluation config from the JSON file

In [3]:
#Create a folder with unix timestamp
root_path = os.getcwd() + '/results/' + str(int(time.time())) + '/'
os.mkdir(root_path)

with open('eval_config.json', 'r') as f:
    eval_config = json.load(f)


print("Evaluation Configuration")
print(json.dumps(eval_config, indent=1))

Evaluation Configuration
{
 "DATASET_PATH": "/scratch/aghosh57/SAT-SAM(Dataset)/ps_rwanda/all_dataset/",
 "DATASET": "RWANDA",
 "MODEL": "SAT-SAM",
 "IMAGE_ENHANCEMENT": "FALSE",
 "FILTRATION": "FALSE",
 "TRAIN_TYPE": "FINETUNE",
 "NMS_THRESHOLD": 0.9,
 "PRED_CONFIDENCE_THRESHOLD": 0.6,
 "ENSEMBLE_BOX_OVERLAP_THRESHOLD": 0.5,
 "ENSEMBLE_BOX_BETA": 0.4,
 "RPN_MODEL_PATH": "rpn/checkpoints/1692043128/rpn_model_1.09.pth",
 "SAM_MODEL_PATH": "sam/checkpoint/sam_vit_l_0b3195.pth"
}


#### Set the device and load the dataset for evaluation

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset_test = PlanetscopeDataset(eval_config['DATASET_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))
# dataset2_test = PlanetscopeDataset(eval_config['DATASET2_PATH'], get_transform(image_enhancement=eval_config['IMAGE_ENHANCEMENT']))

# dataset_test = ConcatDataset([dataset1_test, dataset2_test])

torch.manual_seed(1)
indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[-int(len(indices)*0.2):])

#### Load the proper model to be used for evaluation

In [5]:
if eval_config['MODEL'] == 'SAT-SAM':
    rpn_model = RPN_Model(eval_config['RPN_MODEL_PATH'], 2, device, eval_config['TRAIN_TYPE'])
    sam_model = SAM_Model(eval_config['SAM_MODEL_PATH'], 'large', device)

In [6]:
def pixel_accuracy(predicted, target):
    correct_pixels = np.sum(predicted == target)
    total_pixels = predicted.shape[0] * predicted.shape[1]
    accuracy = correct_pixels / total_pixels
    return accuracy

def f1_score(predicted, target):
    true_positive = np.sum(np.logical_and(predicted == 1, target == 1))
    false_positive = np.sum(np.logical_and(predicted == 1, target == 0))
    false_negative = np.sum(np.logical_and(predicted == 0, target == 1))
    
    precision = true_positive / (true_positive + false_positive + 1e-8)
    recall = true_positive / (true_positive + false_negative + 1e-8)
    
    f1 = 2 * (precision * recall) / (precision + recall + 1e-8)
    return f1

def mean_iou(predicted, target):
    intersection = np.logical_and(predicted, target)
    union = np.logical_or(predicted, target)
    
    intersection_area = np.sum(intersection)
    union_area = np.sum(union)
    
    iou = intersection_area / (union_area + 1e-8)
    return iou

def precision_at_iou_threshold(predicted, target, iou_threshold):
    true_positive = np.sum(np.logical_and(predicted == 1, target == 1))
    false_positive = np.sum(np.logical_and(predicted == 1, target == 0))
    
    iou = true_positive / (true_positive + false_positive + 1e-8)
    
    precision = 0.0
    if iou >= iou_threshold:
        precision = iou
        
    return precision

def collate_mask(masks, img_size):
    cumulative_mask = np.zeros(img_size, dtype=np.uint8)
    
    for mask in masks:
        cumulative_mask[mask > 0] = 1
    
    return cumulative_mask

In [7]:
results = pd.DataFrame(columns=['parcel_id', 'parcel_path', 'gt_mask_ct', 'pred_mask_ct', 'pixel_acc', 'F1_score', 'mIoU'])
    
for i, (id, sam_image, rpn_image, target, ensemble, path)  in enumerate(dataset_test): 
    
    rpn_image = rpn_image.squeeze(0).to(device)  
    predictions = rpn_model.predict(rpn_image)
    predictions = rpn_model.postprocess(predictions, nms_threshold=eval_config['NMS_THRESHOLD'], score_threshold=eval_config['PRED_CONFIDENCE_THRESHOLD'])
    
    if predictions['boxes'].shape[0] == 0:
        print("No boxes detected, generating uniform boxes")
        bboxes = generate_uniform_spaced_bounding_boxes(sam_image.size[0], sam_image.size[1], int(sam_image.size[0]*0.25))
        if eval_config['FILTRATION'] == 'TRUE':
            filtered_boxes = filter_boxes(bboxes, ensemble, eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_boxes)
        else:
            low_res_masks, iou_predictions = sam_model.predict(sam_image, bboxes)
    else:
        if eval_config['FILTRATION'] == 'TRUE':
            filtered_predictions = filter_boxes_predictions(predictions, ensemble, eval_config['ENSEMBLE_BOX_BETA'], eval_config['ENSEMBLE_BOX_OVERLAP_THRESHOLD'])
            low_res_masks, iou_predictions = sam_model.predict(sam_image, filtered_predictions)
        else:
            low_res_masks, iou_predictions = sam_model.predict(sam_image, predictions['boxes'])
    
    high_res_masks = sam_model.postprocess(low_res_masks, tuple(sam_image.size))
    pred_masks = high_res_masks.squeeze().cpu().numpy()

    # Stack the target masks to get a single mask
    pred_masks = collate_mask(pred_masks, sam_image.size)
    target_masks = collate_mask(target['masks'], sam_image.size)

    pix_acc = pixel_accuracy(pred_masks, target_masks)
    f1_sc = f1_score(pred_masks, target_masks) 
    mIoU = mean_iou(pred_masks, target_masks)
    # precision_95 = precision_at_iou_threshold(pred_masks, target_masks, 0.95)

    print("Image Id: ", id)
    print("Pixel Accuracy: ", pix_acc)
    print("F1 Score: ", f1_sc)
    print("mIoU: ", mIoU)
    # print("Precision @ 0.95: ", precision_95)

    results.loc[i] = [id, path, len(target['masks']), len(pred_masks), pix_acc, f1_sc, mIoU]
    # except:
    #     print("Error in image: ", id)
    #     continue

Image Id:  42
Pixel Accuracy:  0.9134695870535714
F1 Score:  0.7099360262693679
mIoU:  0.5503107198341403
Image Id:  14
Pixel Accuracy:  0.5663265306122449
F1 Score:  0.16817981568043147
mIoU:  0.0918102233954766
Image Id:  30
Pixel Accuracy:  0.5502580915178571
F1 Score:  0.18054886877146611
mIoU:  0.09923260385792701
Image Id:  18
Pixel Accuracy:  0.8181002869897959
F1 Score:  0.39793528311545684
mIoU:  0.24838902270803773
Image Id:  67
Pixel Accuracy:  0.8996482382015306
F1 Score:  0.7689960840271529
mIoU:  0.6246902077703113
Image Id:  71
Pixel Accuracy:  0.3054149394132653
F1 Score:  0.04305385706828124
mIoU:  0.02200053317618512
Image Id:  68
Pixel Accuracy:  0.5900231186224489
F1 Score:  0.5155889367677166
mIoU:  0.3473356917365716
Image Id:  27
Pixel Accuracy:  0.903693997130102
F1 Score:  0.24226743132632592
mIoU:  0.13782951960384593
Image Id:  60
Pixel Accuracy:  0.6626275510204082
F1 Score:  0.43623132492405226
mIoU:  0.27896154788143
Image Id:  51
Pixel Accuracy:  0.876021

In [8]:
eval_config['MEAN_IOU'] = results['mIoU'].mean()
eval_config['MEAN_PIXEL_ACCURACY'] = results['pixel_acc'].mean()
eval_config['MEAN_F1_SCORE'] = results['F1_score'].mean()
# eval_config['MEAN_PRECISION@0.95'] = results['precision@0.95'].mean()

In [9]:
results.to_csv(root_path + 'results', index=False)

# save model and configuration
with open(root_path + 'eval_config.json', 'w') as f:
    json.dump(eval_config, f, indent=1)
    f.close()
print("Evaluation Configuration saved to " + root_path + 'train_config.json') 

Evaluation Configuration saved to /home/aghosh57/Kerner-Lab/SAT-SAM/results/1692519291/train_config.json
